In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

# Input file path
path = './kmeans_data/'

# Input data
data = pd.read_csv(path+'data.csv')
X = data.values

In [124]:
k=10

class KMeans:
    def __init__(self, k, distance_function='euclidean', max_iter=100, tol=1e-4):
        self.k = k
        self.distance_function = distance_function
        self.max_iter = max_iter
        self.tol = tol

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def cosine_similarity(self, x1, x2):
        dot_product = np.dot(x1, x2)
        norm_x1 = np.linalg.norm(x1)
        norm_x2 = np.linalg.norm(x2)
        cosine_sim = dot_product / (norm_x1 * norm_x2)
        return 1 - cosine_sim

    def generalized_jaccard_similarity(self, x1, x2):
        intersection = np.sum(np.minimum(x1, x2))
        union = np.sum(np.maximum(x1, x2))
        jaccard_sim = intersection / union
        return 1 - jaccard_sim

    def initialize_centroids(self, X):
        centroids_indices = np.random.choice(X.shape[0], self.k, replace=False)
        centroids = X[centroids_indices]
        return centroids

    def assign_clusters(self, X, centroids):
        distances = np.zeros((X.shape[0], self.k))
        for i in range(self.k):
            if self.distance_function == 'euclidean':
                distances[:, i] = np.array([self.euclidean_distance(X[j], centroids[i]) for j in range(X.shape[0])])
            elif self.distance_function == 'cosine':
                distances[:, i] = np.array([self.cosine_similarity(X[j], centroids[i]) for j in range(X.shape[0])])
            elif self.distance_function == 'jaccard':
                distances[:, i] = np.array([self.generalized_jaccard_similarity(X[j], centroids[i]) for j in range(X.shape[0])])
        cluster_labels = np.argmin(distances, axis=1)
        return cluster_labels

    def update_centroids(self, X, cluster_labels):
        centroids = np.zeros((self.k, X.shape[1]))
        for i in range(self.k):
            cluster_indices = np.where(cluster_labels == i)
            if len(cluster_indices[0]) > 0:
                cluster_data = X[cluster_indices]
                centroids[i] = np.mean(cluster_data, axis=0)
        return centroids

    def fit(self, X):
        centroids = self.initialize_centroids(X)
        for i in range(self.max_iter):
            prev_centroids = centroids
            cluster_labels = self.assign_clusters(X, centroids)
            centroids = self.update_centroids(X, cluster_labels)
            if np.sum(np.abs(centroids - prev_centroids)) < self.tol:
                break
        return cluster_labels, centroids

    def plot_clusters(self, X, cluster_labels, centroids):
        plt.scatter(X[:, 0], X[:, 1], c=cluster_labels, cmap='viridis')
        plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x')
        plt.xlabel('X1')
        plt.ylabel('X2')
        plt.title('K-means Clustering')
        plt.show()

        
kmeans = KMeans(k=k, distance_function='euclidean', max_iter=100)

# Fit the data
#cluster_labels, centroids = kmeans.fit(X)

# Plot the clusters and centroids
#kmeans.plot_clusters(X, cluster_labels, centroids)

In [74]:
# Task 1 - 1 Kmeans Clustering

class KMeans_clustering:
    def __init__(self, k, similarity='euclidean', max_iter=100, tol=1e-4):
        self.k = k
        self.similarity = similarity
        self.max_iter = max_iter
        self.tol = tol
        self.centroids = None

    def fit(self, X):
        # Randomly initialize centroids
        centroids = X[np.random.choice(X.shape[0], size=self.k, replace=False)]

        for _ in range(self.max_iter):
            # Compute similarity between data points and centroids
            if self.similarity == 'euclidean':
                distances = self.euclidean_distance(X, centroids)
            elif self.similarity == 'cosine':
                distances = self.cosine_similarity(X, centroids)
            elif self.similarity == 'jaccard':
                distances = self.jaccard_similarity(X, centroids)
            else:
                raise ValueError('Invalid similarity metric. Choose from "euclidean", "cosine", or "jaccard".')

            # Assign data points to closest centroids
            cluster_labels = np.argmin(distances, axis=1)

            # Update centroids
            new_centroids = np.array([X[cluster_labels == i].mean(axis=0) for i in range(self.k)])

            # Check for convergence
            if np.linalg.norm(new_centroids - centroids) < self.tol:
                break

            centroids = new_centroids

        self.centroids = centroids
        return cluster_labels, centroids

    def euclidean_distance(self, X, centroids):
        return np.sqrt(np.sum((X[:, np.newaxis] - centroids) ** 2, axis=2))

    def cosine_similarity(self, X, centroids):
        X_norm = np.linalg.norm(X, axis=1, keepdims=True)
        centroids_norm = np.linalg.norm(centroids, axis=1, keepdims=True)
        return 1 - np.dot(X, centroids.T) / (X_norm * centroids_norm.T)

    def jaccard_similarity(self, X, centroids):
        intersection = np.dot(X, centroids.T)
        union = np.sum(X, axis=1, keepdims=True) + np.sum(centroids, axis=1) - intersection
        return 1 - intersection / union

    def sum_of_squared_errors(self, X, cluster_labels, centroids):
        return np.sum((X - centroids[cluster_labels]) ** 2)

# Generate sample data
np.random.seed(42)
#X = np.random.rand(100, 2)

# Set number of clusters (k) equal to the number of unique categorical values in y
y = np.random.randint(0, 3, size=100)  # Example categorical labels
k = 10

# Initialize and fit K-Means models with different similarity metrics
kmeans_euclidean = KMeans_clustering(k=k, similarity='euclidean')
cluster_labels_euclidean, centroids_euclidean = kmeans_euclidean.fit(X)
sse_euclidean = kmeans_euclidean.sum_of_squared_errors(X, cluster_labels_euclidean, centroids_euclidean)

kmeans_cosine = KMeans_clustering(k=k, similarity='cosine')
cluster_labels_cosine, centroids_cosine = kmeans_cosine.fit(X)
sse_cosine = kmeans_cosine.sum_of_squared_errors(X, cluster_labels_cosine, centroids_cosine)

kmeans_jaccard = KMeans_clustering(k=k, similarity='jaccard')
cluster_labels_jaccard, centroids_jaccard = kmeans_jaccard.fit(X)
sse_jaccard = kmeans_jaccard.sum_of_squared_errors(X, cluster_labels_jaccard, centroids_jaccard)

print("Sum of Squared Errors (SSE) for Euclidean-K-means:", sse_euclidean)
print("Sum of Squared Errors (SSE) for Cosine-K-means:", sse_cosine)
print("Sum of Squared Errors (SSE) for Jaccard-K-means:", sse_jaccard)

#if sse_euclidean < sse_cosine and sse_euclidean < sse_jaccard:
#    print("Euclidean-K-means has the lowest SSE and is the better method.")
#    elif sse_cosine < sse_euclidean and sse_cosine < sse_jaccard:
#    print("Cosine-K-means has the lowest SSE and is the better method.")
#    else:
#    print("Jaccard-K-means has the lowest SSE and is the better method.")

Sum of Squared Errors (SSE) for Euclidean-K-means: 25405049878.59018
Sum of Squared Errors (SSE) for Cosine-K-means: 25416869026.07649
Sum of Squared Errors (SSE) for Jaccard-K-means: 25412506666.600773


In [47]:
# Task 1 - 2 Compare the accuracies of Euclidean-K-means Cosine-K-means, Jaccard-K-means

class KMeans_clustering:
    def __init__(self, k, similarity='euclidean', max_iter=100, tol=1e-4):
        self.k = k
        self.similarity = similarity
        self.max_iter = max_iter
        self.tol = tol
        self.centroids = None

    def fit(self, X):
        # Randomly initialize centroids
        centroids = X[np.random.choice(X.shape[0], size=self.k, replace=False)]

        for _ in range(self.max_iter):
            # Compute similarity between data points and centroids
            if self.similarity == 'euclidean':
                distances = self.euclidean_distance(X, centroids)
            elif self.similarity == 'cosine':
                distances = self.cosine_similarity(X, centroids)
            elif self.similarity == 'jaccard':
                distances = self.jaccard_similarity(X, centroids)
            else:
                raise ValueError('Invalid similarity metric. Choose from "euclidean", "cosine", or "jaccard".')

            # Assign data points to closest centroids
            cluster_labels = np.argmin(distances, axis=1)

            # Update centroids
            new_centroids = np.array([X[cluster_labels == i].mean(axis=0) for i in range(self.k)])

            # Check for convergence
            if np.linalg.norm(new_centroids - centroids) < self.tol:
                break

            centroids = new_centroids

        self.centroids = centroids
        return cluster_labels, centroids

    def euclidean_distance(self, X, centroids):
        return np.sqrt(np.sum((X[:, np.newaxis] - centroids) ** 2, axis=2))

    def cosine_similarity(self, X, centroids):
        X_norm = np.linalg.norm(X, axis=1, keepdims=True)
        centroids_norm = np.linalg.norm(centroids, axis=1, keepdims=True)
        return 1 - np.dot(X, centroids.T) / (X_norm * centroids_norm.T)

    def jaccard_similarity(self, X, centroids):
        intersection = np.dot(X, centroids.T)
        union = np.sum(X, axis=1, keepdims=True) + np.sum(centroids, axis=1) - intersection
        return 1 - intersection / union

    def sum_of_squared_errors(self, X, cluster_labels, centroids):
        return np.sum((X - centroids[cluster_labels]) ** 2)

    def majority_vote_label(self, X, cluster_labels, y):
        majority_vote_labels = []
        for i in range(self.k):
            cluster_indices = np.where(cluster_labels == i)[0]
            #cluster_labels = y[cluster_indices]           
            unique_labels, counts = np.unique(cluster_labels, return_counts=True)
            majority_vote_label = unique_labels[np.argmax(counts)]
            majority_vote_labels.append(majority_vote_label)
        return majority_vote_labels
    
    def compute_predictive_accuracy(self, X, cluster_labels, majority_vote_labels, y):
        correct_predictions = 0
        for i in range(self.k):
            cluster_indices = np.where(cluster_labels == i)[0]
            cluster_labels = y[cluster_indices]
            correct_predictions += np.sum(cluster_labels == majority_vote_labels[i])
        return correct_predictions / len(y)
    
# Generate sample data
#np.random.seed(42)
#X = np.random.rand(100, 2)

# Set number of clusters (k) equal to the number of unique categorical values in y
y = np.random.randint(0, 10, size=10000)  # Example categorical labels
k = 10

# Initialize and fit KMeans clustering with different similarity metrics
kmeans_euclidean = KMeans_clustering(k, similarity='euclidean')
kmeans_cosine = KMeans_clustering(k, similarity='cosine')
kmeans_jaccard = KMeans_clustering(k, similarity='jaccard')

# Fit KMeans clustering with different similarity metrics
cluster_labels_euclidean, centroids_euclidean = kmeans_euclidean.fit(X)
cluster_labels_cosine, centroids_cosine = kmeans_cosine.fit(X)
cluster_labels_jaccard, centroids_jaccard = kmeans_jaccard.fit(X)

# Compute the accuracy of the clustering
majority_vote_labels_euclidean = kmeans_euclidean.majority_vote_label(X, cluster_labels_euclidean, y)
majority_vote_labels_cosine = kmeans_cosine.majority_vote_label(X, cluster_labels_cosine, y)
majority_vote_labels_jaccard = kmeans_jaccard.majority_vote_label(X, cluster_labels_jaccard, y)

accuracy_euclidean = kmeans_euclidean.compute_predictive_accuracy(X, cluster_labels_euclidean, majority_vote_labels_euclidean, y)
accuracy_cosine = kmeans_cosine.compute_predictive_accuracy(X, cluster_labels_cosine, majority_vote_labels_cosine, y)
accuracy_jaccard = kmeans_jaccard.compute_predictive_accuracy(X, cluster_labels_jaccard, majority_vote_labels_jaccard, y)

print('Accuracy of Euclidean KMeans clustering: ', accuracy_euclidean)
print('Accuracy of Cosine KMeans clustering: ', accuracy_cosine)
print('Accuracy of Jaccard KMeans clustering: ', accuracy_jaccard)
print('=======================================================')
print('Accuracy of Euclidean KMeans clustering: {:.2f}%'.format(accuracy_euclidean * 100))
print('Accuracy of Cosine KMeans clustering: {:.2f}%'.format(accuracy_cosine * 100))
print('Accuracy of Jaccard KMeans clustering: {:.2f}%'.format(accuracy_jaccard * 100))

Accuracy of Euclidean KMeans clustering:  0.0094
Accuracy of Cosine KMeans clustering:  0.0108
Accuracy of Jaccard KMeans clustering:  0.009
Accuracy of Euclidean KMeans clustering: 0.94%
Accuracy of Cosine KMeans clustering: 1.08%
Accuracy of Jaccard KMeans clustering: 0.90%


In [61]:
# Task 1 - 3 Set up the same stop criteria: 
# when there is no change in centroid position OR when the SSE value increases in the next iteration 
# OR when the maximum preset value (e.g., 500, you can set the preset value by yourself) of iteration is complete”

import time

def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

def cosine_similarity(x1, x2):
    dot_product = np.dot(x1, x2)
    norm_x1 = np.linalg.norm(x1)
    norm_x2 = np.linalg.norm(x2)
    return dot_product / (norm_x1 * norm_x2)

def jaccard_similarity(x1, x2):
    intersection = np.sum(np.minimum(x1, x2))
    union = np.sum(np.maximum(x1, x2))
    return intersection / union

def kmeans(data, k, distance_metric, stop_criteria, max_iterations=500):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    iterations = 0
    prev_centroids = None
    prev_sse = None  # Initialize prev_sse
    while iterations < max_iterations:
        distances = np.zeros((n_samples, k))
        for i in range(k):
            if distance_metric == 'euclidean':
                distances[:, i] = np.array([euclidean_distance(data[j], centroids[i]) for j in range(n_samples)])
            elif distance_metric == 'cosine':
                distances[:, i] = np.array([cosine_similarity(data[j], centroids[i]) for j in range(n_samples)])
            elif distance_metric == 'jaccard':
                distances[:, i] = np.array([jaccard_similarity(data[j], centroids[i]) for j in range(n_samples)])
        cluster_assignments = np.argmin(distances, axis=1)
        if prev_centroids is not None and np.array_equal(prev_centroids, centroids):
            print('Converged due to no change in centroid position.')
            break
        prev_centroids = np.copy(centroids)
        centroids = np.array([np.mean(data[cluster_assignments == i], axis=0) for i in range(k)])
        sse = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])
        if stop_criteria == 'sse_increase':
            if prev_sse is not None and sse > prev_sse:
                print('Converged due to SSE increase.')
                break
            prev_sse = sse
        iterations += 1
    else:
        print('Converged due to reaching maximum iterations.')
    return cluster_assignments, centroids, iterations

# Example usage
#data = np.random.rand(100, 3)  # Generate random data with 100 samples and 3 features
k = 10  # Number of clusters
max_iterations = 500  # Maximum number of iterations

# Perform Euclidean-K-means
start_time = time.time()
cluster_assignments_euclidean, centroids_euclidean, iterations_euclidean = kmeans(data, k, 'euclidean', 'sse_increase', max_iterations)
end_time = time.time()
elapsed_time_euclidean = end_time - start_time

# Perform Cosine-Kmeans
start_time = time.time()
cluster_assignments_cosine, centroids_cosine, iterations_cosine = kmeans(data, k, 'cosine', 'sse_increase', max_iterations)
end_time = time.time()
elapsed_time_cosine = end_time - start_time

# Perform Jaccard-K-means
start_time = time.time()
cluster_assignments_jaccard, centroids_jaccard, iterations_jaccard = kmeans(data, k, 'jaccard', 'sse_increase', max_iterations)
end_time = time.time()
elapsed_time_jaccard = end_time - start_time

print(f"Euclidean-K-means converged in {iterations_euclidean} iterations with elapsed time of {elapsed_time_euclidean:.4f} seconds.")
print(f"Cosine-Kmeans converged in {iterations_cosine} iterations with elapsed time of {elapsed_time_cosine:.4f} seconds.")
print(f"Jaccard-K-means converged in {iterations_jaccard} iterations with elapsed time of {elapsed_time_jaccard:.4f} seconds.")

# Compare which method requires more iterations and time to converge
if iterations_euclidean > iterations_cosine and iterations_euclidean > iterations_jaccard:
    print("Euclidean-K-means requires more iterations to converge.")
elif iterations_cosine > iterations_euclidean and iterations_cosine > iterations_jaccard:
    print("Cosine-Kmeans requires more iterations to converge.")
elif iterations_jaccard > iterations_euclidean and iterations_jaccard > iterations_cosine:
    print("Jaccard-K-means requires more iterations to converge.")
else:
    print("The methods have similar number of iterations to converge.")

if elapsed_time_euclidean > elapsed_time_cosine and elapsed_time_euclidean > elapsed_time_jaccard:
    print("Euclidean-K-means takes more time to converge.")
elif elapsed_time_cosine > elapsed_time_euclidean and elapsed_time_cosine > elapsed_time_jaccard:
    print("Cosine-Kmeans takes more time to converge.")
elif elapsed_time_jaccard > elapsed_time_euclidean and elapsed_time_jaccard > elapsed_time_cosine:
    print("Jaccard-K-means takes more time to converge.")
else:
    print("The methods have similar convergence time.")

Converged due to no change in centroid position.
Converged due to SSE increase.
Converged due to SSE increase.
Euclidean-K-means converged in 6 iterations with elapsed time of 0.0837 seconds.
Cosine-Kmeans converged in 1 iterations with elapsed time of 0.0279 seconds.
Jaccard-K-means converged in 1 iterations with elapsed time of 0.0199 seconds.
Euclidean-K-means requires more iterations to converge.
Euclidean-K-means takes more time to converge.


In [75]:
# Task 1 - 4 Set up the same stop criteria.
# Compare the SSEs of Euclidean-K-means Cosine-K-means, Jaccard-K-means with respect to the following three terminating conditions: (10 points)
#  when there is no change in centroid position
#  when the SSE value increases in the next iteration
#  when the maximum preset value (e.g., 100) of iteration is complete

def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

def cosine_similarity(x1, x2):
    dot_product = np.dot(x1, x2)
    norm_x1 = np.linalg.norm(x1)
    norm_x2 = np.linalg.norm(x2)
    return dot_product / (norm_x1 * norm_x2)

def jaccard_similarity(x1, x2):
    intersection = np.sum(np.minimum(x1, x2))
    union = np.sum(np.maximum(x1, x2))
    return intersection / union

def kmeans(data, k, distance_metric, stop_criteria, max_iterations=500):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    iterations = 0
    prev_centroids = None
    prev_sse = None  # Initialize prev_sse
    while iterations < max_iterations:
        distances = np.zeros((n_samples, k))
        for i in range(k):
            if distance_metric == 'euclidean':
                distances[:, i] = np.array([euclidean_distance(data[j], centroids[i]) for j in range(n_samples)])
            elif distance_metric == 'cosine':
                distances[:, i] = np.array([cosine_similarity(data[j], centroids[i]) for j in range(n_samples)])
            elif distance_metric == 'jaccard':
                distances[:, i] = np.array([jaccard_similarity(data[j], centroids[i]) for j in range(n_samples)])
        cluster_assignments = np.argmin(distances, axis=1)
        if prev_centroids is not None and np.array_equal(prev_centroids, centroids):
            print('Converged due to no change in centroid position.')
            break
        prev_centroids = np.copy(centroids)
        centroids = np.array([np.mean(data[cluster_assignments == i], axis=0) for i in range(k)])
        sse = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])
        if stop_criteria == 'sse_increase':
            if prev_sse is not None and sse > prev_sse:
                print('Converged due to SSE increase.')
                break
            prev_sse = sse
        iterations += 1
    else:
        print('Converged due to reaching maximum iterations.')
    return cluster_assignments, centroids, iterations

k = 10
max_iterations = 100

# Stop criteria 1: No change in centroid position
distance_metric = 'euclidean'
cluster_assignments, centroids, iterations = kmeans(data, k, distance_metric, stop_criteria='no_change', max_iterations=max_iterations)
sse_euclidean_no_change = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'cosine'
cluster_assignments, centroids, iterations = kmeans(data, k, distance_metric, stop_criteria='no_change', max_iterations=max_iterations)
sse_cosine_no_change = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'jaccard'
cluster_assignments, centroids, iterations = kmeans(data, k, distance_metric, stop_criteria='no_change', max_iterations=max_iterations)
sse_jaccard_no_change = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

# Stop criteria 2: SSE increase in next iteration
distance_metric = 'euclidean'
cluster_assignments, centroids, iterations = kmeans(data, k, distance_metric, stop_criteria='sse_increase', max_iterations=max_iterations)
sse_euclidean_sse_increase = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'cosine'
cluster_assignments, centroids, iterations = kmeans(data, k, distance_metric, stop_criteria='sse_increase', max_iterations=max_iterations)
sse_cosine_sse_increase = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'jaccard'
cluster_assignments, centroids, iterations = kmeans(data, k, distance_metric, stop_criteria='sse_increase', max_iterations=max_iterations)
sse_jaccard_sse_increase = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

# Stop criteria 3: Maximum preset iterations
distance_metric = 'euclidean'
cluster_assignments, centroids, iterations = kmeans(data, k, distance_metric, stop_criteria='max_iterations', max_iterations=max_iterations)
sse_euclidean_max_iterations = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'cosine'
cluster_assignments, centroids, iterations = kmeans(data, k, distance_metric, stop_criteria='max_iterations', max_iterations=max_iterations)
sse_cosine_max_iterations = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'jaccard'
cluster_assignments, centroids, iterations = kmeans(data, k, distance_metric, stop_criteria='max_iterations', max_iterations=max_iterations)
sse_jaccard_max_iterations = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

# Print SSE results
print("SSE for Euclidean K-means with no change in centroid position: ", sse_euclidean_no_change)
print("SSE for Cosine K-means with no change in centroid position: ", sse_cosine_no_change)
print("SSE for Jaccard K-means with no change in centroid position: ", sse_jaccard_no_change)
print("==============================================================")
print("SSE for Euclidean K-means with SSE increase in next iteration: ", sse_euclidean_sse_increase)
print("SSE for Cosine K-means with SSE increase in next iteration: ", sse_cosine_sse_increase)
print("SSE for Jaccard K-means with SSE increase in next iteration: ", sse_jaccard_sse_increase)
print("==============================================================")
print("SSE for Euclidean K-means with maximum preset iterations: ", sse_euclidean_max_iterations)
print("SSE for Cosine K-means with maximum preset iterations: ", sse_cosine_max_iterations)
print("SSE for Jaccard K-means with maximum preset iterations: ", sse_jaccard_max_iterations)

Converged due to no change in centroid position.
Converged due to reaching maximum iterations.
Converged due to reaching maximum iterations.
Converged due to no change in centroid position.
Converged due to SSE increase.
Converged due to SSE increase.
Converged due to no change in centroid position.
Converged due to reaching maximum iterations.
Converged due to reaching maximum iterations.
SSE for Euclidean K-means with no change in centroid position:  5.79611202532371
SSE for Cosine K-means with no change in centroid position:  26.686480235214347
SSE for Jaccard K-means with no change in centroid position:  26.686480235214347
SSE for Euclidean K-means with SSE increase in next iteration:  4.638807074471246
SSE for Cosine K-means with SSE increase in next iteration:  26.686480235214347
SSE for Jaccard K-means with SSE increase in next iteration:  26.686480235214347
SSE for Euclidean K-means with maximum preset iterations:  4.924714668648514
SSE for Cosine K-means with maximum preset it

In [123]:
# Version 2 ###Task 1 - 4 Set up the same stop criteria.
# Compare the SSEs of Euclidean-K-means Cosine-K-means, Jaccard-K-means with respect to the following three terminating conditions: (10 points)
#  when there is no change in centroid position
#  when the SSE value increases in the next iteration
#  when the maximum preset value (e.g., 100) of iteration is complete


k = 10
max_iterations = 100

def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

def cosine_similarity(x1, x2):
    dot_product = np.dot(x1, x2)
    norm_x1 = np.linalg.norm(x1)
    norm_x2 = np.linalg.norm(x2)
    return dot_product / (norm_x1 * norm_x2)

def jaccard_similarity(x1, x2):
    intersection = np.sum(np.minimum(x1, x2))
    union = np.sum(np.maximum(x1, x2))
    return intersection / union

def kmeans1(data, k, distance_metric, stop_criteria, max_iterations=500):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    iterations = 0
    prev_centroids = None
    prev_sse = None  # Initialize prev_sse
    while iterations < max_iterations:
        distances = np.zeros((n_samples, k))
        for i in range(k):
            if distance_metric == 'euclidean':
                distances[:, i] = np.array([euclidean_distance(data[j], centroids[i]) for j in range(n_samples)])
            elif distance_metric == 'cosine':
                distances[:, i] = np.array([cosine_similarity(data[j], centroids[i]) for j in range(n_samples)])
            elif distance_metric == 'jaccard':
                distances[:, i] = np.array([jaccard_similarity(data[j], centroids[i]) for j in range(n_samples)])
        cluster_assignments = np.argmin(distances, axis=1)
        if prev_centroids is not None and np.array_equal(prev_centroids, centroids):
            print('Converged due to no change in centroid position.')
            break
        prev_centroids = np.copy(centroids)
        centroids = np.array([np.mean(data[cluster_assignments == i], axis=0) for i in range(k)])
#        sse = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])
#        if stop_criteria == 'sse_increase':
#            if prev_sse is not None and sse > prev_sse:
#                print('Converged due to SSE increase.')
#                break
#            prev_sse = sse
        iterations += 1
#    else:
#        print('Converged due to reaching maximum iterations.')
    return cluster_assignments, centroids, iterations

def kmeans2(data, k, distance_metric, stop_criteria, max_iterations=500):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    iterations = 0
    prev_centroids = None
    prev_sse = None  # Initialize prev_sse
    while iterations < max_iterations:
        distances = np.zeros((n_samples, k))
        for i in range(k):
            if distance_metric == 'euclidean':
                distances[:, i] = np.array([euclidean_distance(data[j], centroids[i]) for j in range(n_samples)])
            elif distance_metric == 'cosine':
                distances[:, i] = np.array([cosine_similarity(data[j], centroids[i]) for j in range(n_samples)])
            elif distance_metric == 'jaccard':
                distances[:, i] = np.array([jaccard_similarity(data[j], centroids[i]) for j in range(n_samples)])
        cluster_assignments = np.argmin(distances, axis=1)
#        if prev_centroids is not None and np.array_equal(prev_centroids, centroids):
#            print('Converged due to no change in centroid position.')
#            break
        prev_centroids = np.copy(centroids)
        centroids = np.array([np.mean(data[cluster_assignments == i], axis=0) for i in range(k)])
        sse = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])
        if stop_criteria == 'sse_increase':
            if prev_sse is not None and sse > prev_sse:
                print('Converged due to SSE increase.')
                break
            prev_sse = sse
        iterations += 1
#    else:
#        print('Converged due to reaching maximum iterations.')
    return cluster_assignments, centroids, iterations


def kmeans3(data, k, distance_metric, stop_criteria, max_iterations=500):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    iterations = 0
    prev_centroids = None
    prev_sse = None  # Initialize prev_sse
    while iterations < max_iterations:
        distances = np.zeros((n_samples, k))
        for i in range(k):
            if distance_metric == 'euclidean':
                distances[:, i] = np.array([euclidean_distance(data[j], centroids[i]) for j in range(n_samples)])
            elif distance_metric == 'cosine':
                distances[:, i] = np.array([cosine_similarity(data[j], centroids[i]) for j in range(n_samples)])
            elif distance_metric == 'jaccard':
                distances[:, i] = np.array([jaccard_similarity(data[j], centroids[i]) for j in range(n_samples)])
        cluster_assignments = np.argmin(distances, axis=1)
 #       if prev_centroids is not None and np.array_equal(prev_centroids, centroids):
#            print('Converged due to no change in centroid position.')
#            break
        prev_centroids = np.copy(centroids)
        centroids = np.array([np.mean(data[cluster_assignments == i], axis=0) for i in range(k)])
 #       sse = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])
#        if stop_criteria == 'sse_increase':
#            if prev_sse is not None and sse > prev_sse:
#                print('Converged due to SSE increase.')
#                break
#            prev_sse = sse
        iterations += 1
    else:
        print('Converged due to reaching maximum iterations.')
    return cluster_assignments, centroids, iterations


k = 10
max_iterations = 100

# Stop criteria 1: No change in centroid position
distance_metric = 'euclidean'
cluster_assignments, centroids, iterations = kmeans1(data, k, distance_metric, stop_criteria='no_change', max_iterations=max_iterations)
sse_euclidean_no_change = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'cosine'
cluster_assignments, centroids, iterations = kmeans1(data, k, distance_metric, stop_criteria='no_change', max_iterations=max_iterations)
sse_cosine_no_change = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'jaccard'
cluster_assignments, centroids, iterations = kmeans1(data, k, distance_metric, stop_criteria='no_change', max_iterations=max_iterations)
sse_jaccard_no_change = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

# Stop criteria 2: SSE increase in next iteration
distance_metric = 'euclidean'
cluster_assignments, centroids, iterations = kmeans2(data, k, distance_metric, stop_criteria='sse_increase', max_iterations=max_iterations)
sse_euclidean_sse_increase = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'cosine'
cluster_assignments, centroids, iterations = kmeans2(data, k, distance_metric, stop_criteria='sse_increase', max_iterations=max_iterations)
sse_cosine_sse_increase = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'jaccard'
cluster_assignments, centroids, iterations = kmeans2(data, k, distance_metric, stop_criteria='sse_increase', max_iterations=max_iterations)
sse_jaccard_sse_increase = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

# Stop criteria 3: Maximum preset iterations
distance_metric = 'euclidean'
cluster_assignments, centroids, iterations = kmeans3(data, k, distance_metric, stop_criteria='max_iterations', max_iterations=max_iterations)
sse_euclidean_max_iterations = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'cosine'
cluster_assignments, centroids, iterations = kmeans3(data, k, distance_metric, stop_criteria='max_iterations', max_iterations=max_iterations)
sse_cosine_max_iterations = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

distance_metric = 'jaccard'
cluster_assignments, centroids, iterations = kmeans3(data, k, distance_metric, stop_criteria='max_iterations', max_iterations=max_iterations)
sse_jaccard_max_iterations = np.sum([np.sum((data[cluster_assignments == i] - centroids[i]) ** 2) for i in range(k)])

# Print SSE results
print("SSE for Euclidean K-means with no change in centroid position: ", sse_euclidean_no_change)
print("SSE for Cosine K-means with no change in centroid position: ", sse_cosine_no_change)
print("SSE for Jaccard K-means with no change in centroid position: ", sse_jaccard_no_change)
print("==============================================================")
print("SSE for Euclidean K-means with SSE increase in next iteration: ", sse_euclidean_sse_increase)
print("SSE for Cosine K-means with SSE increase in next iteration: ", sse_cosine_sse_increase)
print("SSE for Jaccard K-means with SSE increase in next iteration: ", sse_jaccard_sse_increase)
print("==============================================================")
print("SSE for Euclidean K-means with maximum preset iterations: ", sse_euclidean_max_iterations)
print("SSE for Cosine K-means with maximum preset iterations: ", sse_cosine_max_iterations)
print("SSE for Jaccard K-means with maximum preset iterations: ", sse_jaccard_max_iterations)

Converged due to no change in centroid position.
Converged due to SSE increase.
Converged due to SSE increase.
Converged due to reaching maximum iterations.
Converged due to reaching maximum iterations.
Converged due to reaching maximum iterations.
SSE for Euclidean K-means with no change in centroid position:  5.174175631895551
SSE for Cosine K-means with no change in centroid position:  26.686480235214347
SSE for Jaccard K-means with no change in centroid position:  26.686480235214347
SSE for Euclidean K-means with SSE increase in next iteration:  5.281121840725107
SSE for Cosine K-means with SSE increase in next iteration:  26.686480235214347
SSE for Jaccard K-means with SSE increase in next iteration:  26.686480235214347
SSE for Euclidean K-means with maximum preset iterations:  4.886735237015621
SSE for Cosine K-means with maximum preset iterations:  26.686480235214347
SSE for Jaccard K-means with maximum preset iterations:  26.686480235214347
